<a href="https://colab.research.google.com/github/jvellayan/Car-crashes-in-VA/blob/main/JUSTSVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data Loading

In [ ]:

import pandas as pd
from google.colab import drive
import numpy as np
np.random.seed(42)
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

path = "/content/drive/My Drive/data/Virginia_Crashes.csv"
crash_df = pd.read_csv(path)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3,6,7,19,20,30,31,37,41,42,46,57,58,66) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
type(crash_df)

pandas.core.frame.DataFrame

*Inspect the Data*

In [ ]:
crash_df.head(5)

,X,Y,OBJECTID,Alcohol_Notalcohol,A_Crash,A_People,Belted_Unbelted,Bike_Nonbike,B_Crash,B_People,Carspeedlimit,Collision_Type,Crash_Dt,Crash_Event_Type_Dsc,Crash_Military_Tm,Crash_Severity,Crash_Year,C_Crash,C_People,Deer_Nodeer,Distracted_Notdistracted,Document_Nbr,Driverage,Drivergen,Driverinjurytype,Driver_Action_Type_Cd,FAC,Fatal_Crashes,First_Harmful_Event,FUN,Gr_Nogr,Hitrun_Not_Hitrun,K_People,LATITUDE,Light_Condition,Local_Case_Cd,LONGITUDE,Motor_Nonmotor,MPO_NAME,Ownership,Passage,Passgen,Passinjurytype,Pdo_Crash,Pedage,Pedestrians_Injured,Pedinjurytype,Ped_Nonped,Persons_Injured,Rd_Type,Rns_Mp,Roadway_Alignment,Roadway_Surface_Cond,Route_Or_Street_Nm,Rte_Cat,Rte_Nm,School_Zone,Senior_Notsenior,Speed_Notspeed,Summons_Issued_Cd,SYSTEM,Vehiclenumber,Vehicle_Body_Type_Cd,VSP,Weather_Condition,Work_Zone_Related,Young_Notyoung,Vdot_District,PDCNAME,DIAGRAM,Physical_Juris
0,-79.967048,37.129694,4001,NaN,0,0,NaN,NaN,0,0,NaN,2. Angle,2018-12-22T00:00:00.000Z,20. Motor Vehicle In Transport,1547,PDO.Property Damage Only,2018,0,0,NaN,NaN,183605178,"57,52","Female,Male","PDO,PDO","16. Improper Turn From Wrong Lane,1. No Improp...","1.Divided, no control of access",0,1. On Roadway,2.Rural Other Principal Arterial,NaN,NaN,0,37.129686,2. Daylight,2018-094097,-79.967045,NaN,NaN,VDOT_PRI,NaN,NaN,NaN,1,NaN,0,NaN,NOTPED,0,NOT-RD,51.072294,1. Straight - Level,1. Dry,26472 Virgil H Goode Hwy,US,R-VA US00220NB,3. No,NaN,NaN,"1. Yes,2. No",PRIMARY,"1,2","3. Van,3. Van",6,1. No Adverse Condition (Clear/Cloudy),2. No,NaN,2.Salem District,"Roanoke Valley-Alleghany, West Piedmont",http://rns/APP/Images/CrashDiagramProxy.aspx?d...,33.Franklin
1,-79.342818,37.301027,4002,NaN,0,0,NaN,NaN,0,0,60.0,4. Sideswipe - Same Direction,2018-09-01T00:00:00.000Z,20. Motor Vehicle In Transport,1635,PDO.Property Damage Only,2018,0,0,NaN,NaN,182485223,"51,29","Male,Male","PDO,PDO","14. Improper Turn - Wide Right Turn,1. No Impr...","1.Divided, no control of access",0,1. On Roadway,2.Rural Other Principal Arterial,NaN,NaN,0,37.301020,2. Daylight,201819137,-79.342815,NaN,Central Virginia Metropolitan Planning Organiz...,VDOT_PRI,NaN,NaN,NaN,1,NaN,0,NaN,NOTPED,0,NOT-RD,193.905648,1. Straight - Level,1. Dry,460/E LYN SALEM TURNPIKE,US,R-VA US00460WB,3. No,NaN,NaN,"2. No,2. No",PRIMARY,"1,2","2. Truck - Pick-up/Passenger Truck,1. Passenge...",6,1. No Adverse Condition (Clear/Cloudy),2. No,NaN,2.Salem District,Region 2000 Regional,http://rns/APP/Images/CrashDiagramProxy.aspx?d...,9.Bedford
2,-79.497418,37.323679,4003,NaN,0,0,NaN,NaN,0,0,50.0,1. Rear End,2015-04-25T00:00:00.000Z,20. Motor Vehicle In Transport,2119,PDO.Property Damage Only,2015,0,0,NaN,NaN,151165237,"22,45","Female,Female","PDO,PDO","37. Other,1. No Improper Action","1.Divided, no control of access",0,1. On Roadway,E.Urban Other Principal Arterials; Connecting ...,NaN,NaN,0,37.323671,4. Darkness - Road Lighted,2015002033,-79.497415,NaN,NaN,VDOT_PRI,NaN,NaN,NaN,1,NaN,0,NaN,NOTPED,0,NOT-RD,184.677109,1. Straight - Level,2. Wet,E LYNCHBURG SALEM TPKE,US,R-VA US00460EB,3. No,NaN,NaN,"1. Yes,2. No",PRIMARY,"1,2","1. Passenger car,22. Truck - Sport Utility Veh...",6,5. Rain,2. No,NaN,2.Salem District,Region 2000 Regional,http://rns/APP/Images/CrashDiagramProxy.aspx?d...,141.Bedford
3,-79.917538,37.078088,4004,NaN,0,0,NaN,NaN,0,0,55.0,4. Sideswipe - Same Direction,2019-11-24T00:00:00.000Z,20. Motor Vehicle In Transport,1915,PDO.Property Damage Only,2019,0,0,NaN,NaN,193305263,"44,21","Male,Male","PDO,PDO","42. Improper or Unsafe Lane Change,1. No Impro...","1.Divided, no control of access",0,1. On Roadway,E.Urban Other Principal Arterials; Connecting ...,NaN,NaN,0,37.078081,5. Darkness - Road Not Lighted,DIV619106121,-79.917535,NaN,NaN,VDOT_PRI,NaN,NaN,NaN,1,NaN,0,NaN,NOTPED,0,NOT-RD,46.289636,3. Grade - Straight,1. Dry,VIRGIL H. GOODE HIGHWAY (US 220),US,R-VA US00220NB,3. No,NaN,NaN,"1. Yes,2. No",PRIMARY,"1,2","2. Truck - Pick-up/Passenger Truck,1. Passenge...",6,1. No Adverse Condition (Clear/Cloudy),2. No,NaN

In [ ]:
crash_df = crash_df.drop(['X','Y', 'OBJECTID', 'Crash_Event_Type_Dsc','Young_Notyoung','Senior_Notsenior', 'Crash_Year', 'Crash_Dt','Document_Nbr', 'Driverinjurytype','Gr_Nogr', 'Local_Case_Cd', 'MPO_NAME','Ownership', 'Passgen', 'Passinjurytype', 'Pedinjurytype', 'Ped_Nonped','Rns_Mp','Route_Or_Street_Nm','Rte_Cat', 'Rd_Type','Rte_Nm','Summons_Issued_Cd','SYSTEM','Vehiclenumber','VSP', 'Vdot_District', 'PDCNAME','DIAGRAM','Physical_Juris'], axis=1)

# Data Cleaning

Now that we have reduced our dataset to nonrepetive and relevant features, we must address the missingness in our data. Many binary features have NaN instead of zeros. These binary features are named "Alcohol_notAlcohol", "Belted_Unbelted", "Bike_Nonbike", "Deer_Nodeer", "Distracted_Notdistracted", "Motor_Nonmotor", "Senior_Notsenior", "Speed_Notspeed", "Young_Notyoung", and "Hitrun_Not_Hitrun". These features are only filled out when their flag occured; otherwise, they are left blank and appear as NaN in our data. We replaced all of the NaN in the binary features with 0 and changed their flag from a string to 1.

In [ ]:
#Replace the flag for these columns with a 1
crash_df['Alcohol_Notalcohol'] = crash_df['Alcohol_Notalcohol'].replace('ALCOHOL', 1) #1 indicates alcohol 
crash_df['Belted_Unbelted'] = crash_df['Belted_Unbelted'].replace('UNBELTED', 1) #1 indicates unbelted
crash_df['Bike_Nonbike'] = crash_df['Bike_Nonbike'].replace('BIKE', 1) #1 indicates bike
crash_df['Deer_Nodeer'] = crash_df['Deer_Nodeer'].replace('DEER', 1) #1 indicates deer
crash_df['Distracted_Notdistracted'] = crash_df['Distracted_Notdistracted'].replace('DISTRACTED', 1) #1 indicates distracted
crash_df['Motor_Nonmotor'] = crash_df['Motor_Nonmotor'].replace('MOTOR', 1) #1 indicates motor
crash_df['Speed_Notspeed'] = crash_df['Speed_Notspeed'].replace('SPEED', 1) #1 indicates speeding
crash_df['Hitrun_Not_Hitrun'] = crash_df['Hitrun_Not_Hitrun'].replace('HIT_RUN', 1) #1 indicates hit and run

In [ ]:
#Replace the NaN values for these columns with a 0
crash_df['Alcohol_Notalcohol'] = crash_df['Alcohol_Notalcohol'].fillna(0)
crash_df['Belted_Unbelted'] = crash_df['Belted_Unbelted'].fillna(0)
crash_df['Bike_Nonbike'] = crash_df['Bike_Nonbike'].fillna(0)
crash_df['Deer_Nodeer'] = crash_df['Deer_Nodeer'].fillna(0)
crash_df['Distracted_Notdistracted'] = crash_df['Distracted_Notdistracted'].fillna(0)
crash_df['Motor_Nonmotor'] = crash_df['Motor_Nonmotor'].fillna(0)
crash_df['Speed_Notspeed'] = crash_df['Speed_Notspeed'].fillna(0)
crash_df['Hitrun_Not_Hitrun'] = crash_df['Hitrun_Not_Hitrun'].fillna(0)

While the feature "Work_Zone_Related" does not have missingness, it is a binary represented with "Yes" and "No". We will recode that as 1 for "Yes" and 0 for "No".

In [ ]:
crash_df['Carspeedlimit'] = crash_df['Carspeedlimit'].fillna(crash_df['Carspeedlimit'].median())

In [ ]:
crash_df.groupby('Work_Zone_Related').size()

Work_Zone_Related
1. Yes     21592
2. No     862414
dtype: int64

In [ ]:
crash_df['Work_Zone_Related'] = crash_df['Work_Zone_Related'].replace('1. Yes', 1) 
crash_df['Work_Zone_Related'] = crash_df['Work_Zone_Related'].replace('2. No', 0) 

Additionally, the feature "School_Zone" can be turned into a binary with 1 for "Yes" and 0 for "No".

In [ ]:
crash_df.groupby('School_Zone').size()

School_Zone
1. Yes                             9489
2. Yes - With School Activity      3483
3. No                            871034
dtype: int64

In [ ]:
crash_df['School_Zone'] = crash_df['School_Zone'].replace('1. Yes', 1) 
crash_df['School_Zone'] = crash_df['School_Zone'].replace('2. Yes - With School Activity', 1) 
crash_df['School_Zone'] = crash_df['School_Zone'].replace('3. No', 0) 

Additionally, "Passage" and "Pedage" only have values indicating the age of a passenger or a pedestrian if there was one. We will recode these columns to contain an age if there was a passenger or pedestrian and to have 0 if there was not one.

In [ ]:
crash_df['Passage'] = crash_df['Passage'].fillna(-1)
crash_df['Pedage'] = crash_df['Pedage'].fillna(-1)

469,353 of 884,006 instances have a speed limit. We will use median imputation to provide data for the instance without a speed limit.

FAC is also missing values. 784,674 of the 884,006 instances have values. It is a categorical feature, making imputation difficult. The same instance missing the feature "FAC" are also missing the feature "FUN". We will drop those instances, since they are missing both features and they only make up about 11% of the dataset. If our dataset were not so large, we would not do this.

In [ ]:
crash_df = crash_df.dropna(subset=['FAC'])

In the previous dataframe, some "LATITUDE" and "LONGITUDE" values were missing. All of the instances that were missing "FAC" and "FUN" were also missing "LATITUDE" and "LONGITUDE". Dropping those instances leads to no missing values in "FAC", "FUN", "LATITUDE", or "LONGITUDE". Now the only column with missing values is "Driverage"

*Features that still need work:*
*   DriverAge - break into categories number of drivers in each range, numerical features (DONE)
*   DriverGen - break into features number in each (DONE)
*   Driver_Action_Type_Cd - break into features number of each action (DONE)
*   Vehicle_Body_Type - break into features of number of each vehicle involved (DONE)

*Driverage*

In [ ]:
crash_df["Driverage"].value_counts()

19             9948
21             9664
22             9572
20             9292
18             9159
               ... 
75,18,50          1
36,48,26          1
25,41,30,48       1
25,20,64          1
59,48,20          1
Name: Driverage, Length: 70858, dtype: int64

In [ ]:
# first drop the instances where the driver age its null
crash_df = crash_df.dropna(subset=['Driverage']) #will drop ~9,000 values which is about 1%
#still 775,095 instances which is huge

In [ ]:
driverages = crash_df["Driverage"]
#now there are no null values
driverages.isna().sum()


0

In [ ]:
da_list = driverages.to_list() #probably a better way to do this but whatever
n = len(da_list)
#initialize list indices to 0 for different age groups
youngList = [0]*n #ages 15 to 25
midYoung = [0]*n #ages 25 to 40
mid= [0]*n #ages 40-60
midOld = [0]*n #ages 60-75
old =[0]*n #ages 75+
for i in range(len(da_list)):
  string = da_list[i]
  #split the string into a list containing the ages as strings
  subList = string.split(',')

  #each age in the sublist check which range it falls into and increment the index of that ranges element there
  for num in subList:
    if (15 <= int(num) and int(num)< 25):
      youngList[i]+= 1
    elif (25 <= int(num) and int(num)< 40):
      midYoung[i] +=1
    elif (40 <= int(num) and int(num)< 60):
      mid[i] +=1 
    elif (60 <= int(num) and int(num)< 75):
      midOld[i] +=1
    elif (75 <= int(num)):
      old[i] +=1
    else:
      break


In [ ]:
print(youngList[:15])
print(midYoung[:15])
print(mid[:15])
print(da_list[:15])
#seems to be working

[0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0]
[0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1]
[2, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 2, 1, 0, 1]
['57,52', '51,29', '22,45', '44,21', '41,36', '53,37', '30,59', '23,44', '28,23', '30,57', '54,26', '40,41', '22,54', '22,62', '53,38']


In [ ]:
#add these columns to the dataframe
crash_df['drAge_young'] = youngList
crash_df['drAge_midyoung'] = midYoung
crash_df['drAge_mid'] = mid
crash_df['drAge_midold'] = midOld
crash_df['drAge_old'] = old

In [ ]:
#now that I've dealt with it and have condensed these other features I can drop the original
crash_df = crash_df.drop(['Driverage'], axis=1)

DriverGen - driver gender

In [ ]:
drivergens = crash_df["Drivergen"]
dg_list = drivergens.to_list() #probably a better way to do this but whatever
n = len(dg_list)
#initialize list indices to 0 for different age groups
malesInvolved = [0]*n 
femalesInvolved = [0]*n 
notProvided= [0]*n 
for i in range(len(dg_list)):
  string = dg_list[i]
  #print(string)
  #split the string into a list containing the ages as strings
  subList = string.split(',')

  #each age in the sublist check which range it falls into and increment the index of that ranges element there
  for person in subList:
    if (person == 'Male'):
      malesInvolved[i]+= 1
    elif (person == 'Female'):
      femalesInvolved[i] +=1
    else:
      notProvided[i]+=1

In [ ]:
print(malesInvolved[:15])
print(femalesInvolved[:15])
print(notProvided[:15])
print(dg_list[:15])
#seems to be working

[1, 2, 0, 2, 1, 0, 1, 1, 1, 1, 2, 1, 1, 2, 2]
[1, 0, 2, 0, 1, 2, 1, 1, 1, 1, 0, 1, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['Female,Male', 'Male,Male', 'Female,Female', 'Male,Male', 'Male,Female', 'Female,Female', 'Female,Male', 'Male,Female', 'Female,Male', 'Male,Female', 'Male,Male', 'Female,Male', 'Female,Male', 'Male,Male', 'Male,Male']


In [ ]:
#add these columns to the dataframe
crash_df['malesInvolved'] = malesInvolved
crash_df['femalesInvolved'] = femalesInvolved
crash_df['notProvidedInvolved'] = notProvided

In [ ]:
#now i can drop the category
crash_df = crash_df.drop(['Drivergen'], axis=1)

*Driver_Action_Type* 

In [ ]:
crash_df["Driver_Action_Type_Cd"].value_counts()

12. Following Too Close,1. No Improper Action                                                                                                                  136441
40. Fail to Maintain Proper Control                                                                                                                            101464
11. Did Not Have Right-of-Way,1. No Improper Action                                                                                                             91112
1. No Improper Action                                                                                                                                           58539
40. Fail to Maintain Proper Control,1. No Improper Action                                                                                                       35982
                                                                                                                                                                ...  
25. 

In [ ]:
#find all the possible types of actions by using a python set
dActType = crash_df["Driver_Action_Type_Cd"]
possibleActions = set()
dAct_list = dActType.to_list() #probably a better way to do this but whatever
n = len(dAct_list) 
for i in range(len(dAct_list)):
  string = dAct_list[i]
  #print(string)
  #split the string into a list containing the actions as strings
  subList = string.split(',')

  #go through eachtion
  for action in subList:
    possibleActions.add(action) #since its a set it will only add to this if its not already in there
    

actionsList = list(possibleActions)
sortedPossibleActions = sorted(actionsList)
print(sortedPossibleActions)
print(len(actionsList))


['1. No Improper Action', '10. Wrong Side Of Road - Not Overtaking', '11. Did Not Have Right-of-Way', '12. Following Too Close', '13. Fail to Signal or Improper Signal', '14. Improper Turn - Wide Right Turn', '15. Improper Turn - Cut Corner on Left Turn', '16. Improper Turn From Wrong Lane', '17. Other Improper Turn', '18. Improper Backing', '19. Improper Start From Parked Position', '2. Exceeded Speed Limit', '20. Disregarded Officer or Flagger', '21. Disregarded Traffic Signal', '22. Disregarded Stop or Yield Sign', '23. Driver Distraction', '24. Fail to Stop at Through Highway - No Sign', '25. Drive Through Work Zone', '26. Fail to Set Out Flares or Flags', '27. Fail to Dim Headlights', '28. Driving Without Lights', '29. Improper Parking Location', '3. Exceeded Safe Speed But Not Speed Limit', '30. Avoiding Ped', '31. Avoiding Other Vehicle', '32. Avoiding Animal', '33. Crowded Off Highway', '34. Hit and Run', '35. Car Ran Away - No Driver', '36. Blinded by Headlights', '37. Other',

In [ ]:

#now I know the categories, I can go through the dataset and turn each feature into a boolean where its 1 if the thing happened in that incident and 0 if it didn't
#probably easier way to do this but I'll just brute force this
n = len(dAct_list) #basically the number of instances

improperActions = np.zeros([n,44], dtype = int)

for i in range(n): #go through each instance
  string = dAct_list[i]
  #print(string)
  #split the string into a list containing the actions as strings
  subList = string.split(',')
  #print(subList)
  #go through eachtion
  for actionString in subList:
    
    if actionString == 'Not Applicable':
      improperActions[i][0] = 1
    else:
      #split again based on the decimal and just get the incident number
      actionNum= int(actionString.split('.')[0])
      #add it to the corresponding part of improper actions
      improperActions[i][actionNum] = 1 #basically makes the feature a boolean if a particular action happened in a crash

In [ ]:
#turn each column of improperActions array into a feature
actionColList = list(improperActions.T)
for actNum in range(1, len(actionColList)): #iterate through each column
  #print(actNum)
  #print(list(actionColList[actNum]))
  crash_df['ImpropAct'+str(actNum)] = list(actionColList[actNum])

In [ ]:
crash_df = crash_df.drop(['Driver_Action_Type_Cd'], axis=1)

Vehicle_Body_Type

In [ ]:
#find all the possible types of actions by using a python set
vehicleType = crash_df["Vehicle_Body_Type_Cd"]
possibleVehicles = set()
vtype_list = vehicleType.to_list() #probably a better way to do this but whatever
n = len(vtype_list) 
for i in range(len(vtype_list)):
  string = vtype_list[i]
  #print(string)
  #split the string into a list containing the actions as strings
  subList = string.split(',')

  #go through eachtion
  for vehicle in subList:
    possibleVehicles.add(vehicle) #since its a set it will only add to this if its not already in there
    

vehiclesList = list(possibleVehicles)
sortedPossibleVehicles= sorted(vehiclesList)
print(sortedPossibleVehicles)
print(len(vehiclesList))


[' Bookmobile', ' Go-cart', ' Golf Cart', ' Hearse', ' Recreational Vehicle', ' etc.)', '1. Passenger car', '10. Moped', '11. Motorcycle', '12. Emergency Vehicle (Regardless of Veh Type)', '13. Bus - School Bus', '14. Bus - City Transit Bus/Privately Owned Church Bus', '15. Bus - Commercial Bus', '16. Other (Scooter', '18. Special Vehicle - Farm Machinery', '19. Special Vehicle - ATV', '2. Truck - Pick-up/Passenger Truck', '21. Special Vehicle - Low Speed Vehicle', '22. Truck - Sport Utility Vehicle (SUV)', '23. Truck - Single Unit Truck (3 Axles or More)', '25. Truck - Truck Tractor (Bobtail-No Trailer)', '3. Van', '4. Truck - Single Unit Truck (2-Axles)', '7. Motor Home', '8. Special Vehicle - Oversized (Veh/Earthmover/Road Equip.)', '9. Bicycle', 'Not Applicable']
27


In [ ]:
#now I know the categories, I can go through the dataset and turn each feature into a boolean where its 1 if the thing happened in that incident and 0 if it didn't
#probably easier way to do this but I'll just brute force this
n = len(vtype_list) #basically the number of instances
print(n)
vehiclesInvolved = np.zeros([n,len(vehiclesList)-1], dtype = int)

for i in range(n): #go through each instance
  string = vtype_list[i]
  #print(string)
  #split the string into a list containing the actions as strings
  subList = string.split(',')
  #print(subList)
  #go through eachtion
  for vehicleString in subList:
    
    if vehicleString == 'Not Applicable':
      vehiclesInvolved[i][0] = 1
    elif ((vehicleString == ' Bookmobile') or (vehicleString == ' Go-cart') or (vehicleString == ' Golf Cart') or (vehicleString == ' Hearse') or (vehicleString == ' Recreational Vehicle') or (vehicleString == ' etc.)')): #saying these count as other
      vehiclesInvolved[i][16] = 1
    else:
      #split again based on the decimal and just get the incident number
      vehicleNum= int(vehicleString.split('.')[0])
      #add it to the corresponding part of improper actions
      vehiclesInvolved[i][vehicleNum] = 1 #basically makes the feature a boolean if a particular action happened in a crash

775095


In [ ]:
#turn each column of improperActions array into a feature
vehicleColList = list(vehiclesInvolved.T)
for vNum in range(1, len(vehicleColList)): #iterate through each column
  crash_df['Vehicle_Type_'+str(vNum)] = list(vehicleColList[vNum])

In [ ]:
crash_df = crash_df.drop(['Vehicle_Body_Type_Cd'], axis=1)

Fatal Crashes

In [ ]:
crash_df["Fatal_Crashes"].value_counts()

0    770335
1      4760
Name: Fatal_Crashes, dtype: int64

Above is the a plot of fatal crashes that is color coordinated by the speed limit at the crash location.

#Crash-Severity Classification

**CUSTOM CRASH SEVERITY METRIC**

We want our SVM to predict the following 3 categories:
*  MILD - only property damage
*  MEDIUM - Only medium visible- and non- visible injuries occured in a quantity less than 5
*  SEVERE - either there was a severe injury, or the crash was fatal, or 5 or more people were hurt in a medium manner

In [ ]:
crash_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 775095 entries, 0 to 884005
Data columns (total 112 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   Alcohol_Notalcohol        float64
 1   A_Crash                   int64  
 2   A_People                  int64  
 3   Belted_Unbelted           float64
 4   Bike_Nonbike              float64
 5   B_Crash                   int64  
 6   B_People                  int64  
 7   Carspeedlimit             float64
 8   Collision_Type            object 
 9   Crash_Military_Tm         int64  
 10  Crash_Severity            object 
 11  C_Crash                   int64  
 12  C_People                  int64  
 13  Deer_Nodeer               float64
 14  Distracted_Notdistracted  float64
 15  FAC                       object 
 16  Fatal_Crashes             int64  
 17  First_Harmful_Event       object 
 18  FUN                       object 
 19  Hitrun_Not_Hitrun         float64
 20  K_People                 

In [ ]:
#build our custom crash severity metric into the dataset for training
print(n)
severityList= []
for crash in crash_df.index:  #loop through every crash
  #print(crash_df['Name'][crash], crash_df['Stream'][crash]) 
  if((crash_df['K_People'][crash] >= 1) or (crash_df['A_People'][crash] >=1)) :
      severityList.append('SEVERE')
  elif ((crash_df['B_People'][crash] >= 1) or (crash_df['C_People'][crash] >=1)): #if someone was injured in a medium visible or non visible manner
    severityList.append('MEDIUM')
  else:
    #this means only property damage occured
    severityList.append('MILD')

775095


In [ ]:
print(len(severityList))

775095


*Add our Severity Metric to the dataframe and drop the others that could be related to it*

In [ ]:
#add these columns to the dataframe
crash_df['SEVERITY'] = severityList

In [ ]:
#drop the others
crash_df_final = crash_df.drop(["A_Crash", "A_People", "B_Crash", "B_People", "Crash_Severity", "C_Crash", "C_People", "Fatal_Crashes", "K_People", "Persons_Injured","Pedestrians_Injured"], axis=1)

In [ ]:
feature_labels = list(crash_df_final)
feature_labels

['Alcohol_Notalcohol',
 'Belted_Unbelted',
 'Bike_Nonbike',
 'Carspeedlimit',
 'Collision_Type',
 'Crash_Military_Tm',
 'Deer_Nodeer',
 'Distracted_Notdistracted',
 'FAC',
 'First_Harmful_Event',
 'FUN',
 'Hitrun_Not_Hitrun',
 'LATITUDE',
 'Light_Condition',
 'LONGITUDE',
 'Motor_Nonmotor',
 'Passage',
 'Pdo_Crash',
 'Pedage',
 'Roadway_Alignment',
 'Roadway_Surface_Cond',
 'School_Zone',
 'Speed_Notspeed',
 'Weather_Condition',
 'Work_Zone_Related',
 'drAge_young',
 'drAge_midyoung',
 'drAge_mid',
 'drAge_midold',
 'drAge_old',
 'malesInvolved',
 'femalesInvolved',
 'notProvidedInvolved',
 'ImpropAct1',
 'ImpropAct2',
 'ImpropAct3',
 'ImpropAct4',
 'ImpropAct5',
 'ImpropAct6',
 'ImpropAct7',
 'ImpropAct8',
 'ImpropAct9',
 'ImpropAct10',
 'ImpropAct11',
 'ImpropAct12',
 'ImpropAct13',
 'ImpropAct14',
 'ImpropAct15',
 'ImpropAct16',
 'ImpropAct17',
 'ImpropAct18',
 'ImpropAct19',
 'ImpropAct20',
 'ImpropAct21',
 'ImpropAct22',
 'ImpropAct23',
 'ImpropAct24',
 'ImpropAct25',
 'ImpropAct2

In [ ]:
feature_labels_one_hot = ['Alcohol_Notalcohol',
 'Belted_Unbelted',
 'Bike_Nonbike',
 'Carspeedlimit',
'2. Angle', '4. Sideswipe - Same Direction', '1. Rear End',
       '5. Sideswipe - Opposite Direction', '3. Head On',
       '15. Backed Into', '9. Fixed Object - Off Road', '16. Other',
       '8. Non-Collision', '6. Fixed Object in Road', '10. Deer',
       '11. Other Animal', '12. Ped', '14. Motorcyclist', '7. Train',
       '13. Bicyclist', 'Not Provided', 'Not Applicable',
 'Crash_Military_Tm',
 'Deer_Nodeer',
 'Distracted_Notdistracted',
'1.Divided, no control of access',
       '3.Divided, full control of access', '0.Two-way, non-divided',
       '4.One-way, part of a one-way system', '7.Transition',
       '2.Divided, partial control of access',
       '5.Two-way, part of a one-way system',
       'B.Two-way structure (bridge, tunnel, causeway, etc.',
       '6.One-way couplet',
'1. On Roadway', '7. In Parking Lane or Zone', '3. Median',
       '2. Shoulder', '4. Roadside', '8. Off Roadway, Location Unknown',
       '9. Outside Right-of-Way', '5. Gore', '6. Separator',
'2.Rural Other Principal Arterial',
       'E.Urban Other Principal Arterials; Connecting Links of Other Rural Principal Arterial',
       'H.Urban Minor Arterial', '1.Rural Interstate',
       'A.Urban Interstate', '3.Rural Minor Arterial',
       '4.Rural Major Collector', '6.Rural Local', 'I.Urban Collector',
       'J.Urban Local',
       'B.Urban Freeways and Expressways; Connecting Links of Rural Principal Arterial',
       '5.Rural Minor Collector',
 'Hitrun_Not_Hitrun',
 'LATITUDE',
'2. Daylight', '4. Darkness - Road Lighted',
       '5. Darkness - Road Not Lighted', '3. Dusk', '1. Dawn',
       '6. Darkness - Unknown Road Lighting', '7. Unknown',
 'LONGITUDE',
 'Motor_Nonmotor',
 'Passage',
 'Pdo_Crash',
 'Pedage',
'1. Straight - Level', '3. Grade - Straight', '2. Curve - Level',
       '4. Grade - Curve', '6. Hillcrest - Curve', '7. Dip - Straight',
       '10. On/Off Ramp', '5. Hillcrest - Straight', '9. Other',
       '8. Dip - Curve',
'1. Dry', '2. Wet', '7. Other', '3. Snowy', '4. Icy', '10. Slush',
       '9. Water (Standing, Moving)', '6. Oil/Other Fluids',
       '11. Sand, Dirt, Gravel', '8. Natural Debris', '5. Muddy',
 'School_Zone',
 'Speed_Notspeed',
'1. No Adverse Condition (Clear/Cloudy)', '5. Rain', '4. Mist',
       '3. Fog', '6. Snow', '7. Sleet/Hail', '8. Smoke/Dust', '9. Other',
       '11. Severe Crosswinds', '10. Blowing Sand, Soil, Dirt, or Snow',
       'Not Applicable',
 'Work_Zone_Related',
 'drAge_young',
 'drAge_midyoung',
 'drAge_mid',
 'drAge_midold',
 'drAge_old',
 'malesInvolved',
 'femalesInvolved',
 'notProvidedInvolved',
 'ImpropAct1',
 'ImpropAct2',
 'ImpropAct3',
 'ImpropAct4',
 'ImpropAct5',
 'ImpropAct6',
 'ImpropAct7',
 'ImpropAct8',
 'ImpropAct9',
 'ImpropAct10',
 'ImpropAct11',
 'ImpropAct12',
 'ImpropAct13',
 'ImpropAct14',
 'ImpropAct15',
 'ImpropAct16',
 'ImpropAct17',
 'ImpropAct18',
 'ImpropAct19',
 'ImpropAct20',
 'ImpropAct21',
 'ImpropAct22',
 'ImpropAct23',
 'ImpropAct24',
 'ImpropAct25',
 'ImpropAct26',
 'ImpropAct27',
 'ImpropAct28',
 'ImpropAct29',
 'ImpropAct30',
 'ImpropAct31',
 'ImpropAct32',
 'ImpropAct33',
 'ImpropAct34',
 'ImpropAct35',
 'ImpropAct36',
 'ImpropAct37',
 'ImpropAct38',
 'ImpropAct39',
 'ImpropAct40',
 'ImpropAct41',
 'ImpropAct42',
 'ImpropAct43',
 'Vehicle_Type_1',
 'Vehicle_Type_2',
 'Vehicle_Type_3',
 'Vehicle_Type_4',
 'Vehicle_Type_5',
 'Vehicle_Type_6',
 'Vehicle_Type_7',
 'Vehicle_Type_8',
 'Vehicle_Type_9',
 'Vehicle_Type_10',
 'Vehicle_Type_11',
 'Vehicle_Type_12',
 'Vehicle_Type_13',
 'Vehicle_Type_14',
 'Vehicle_Type_15',
 'Vehicle_Type_16',
 'Vehicle_Type_17',
 'Vehicle_Type_18',
 'Vehicle_Type_19',
 'Vehicle_Type_20',
 'Vehicle_Type_21',
 'Vehicle_Type_22',
 'Vehicle_Type_23',
 'Vehicle_Type_24',
 'Vehicle_Type_25']

In [ ]:
len(feature_labels_one_hot)

180

In [ ]:
crash_df_subset = crash_df_final.sample(frac = .2)
crash_df_subset.info(verbose='true')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 155019 entries, 758908 to 14714
Data columns (total 102 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   Alcohol_Notalcohol        float64
 1   Belted_Unbelted           float64
 2   Bike_Nonbike              float64
 3   Carspeedlimit             float64
 4   Collision_Type            object 
 5   Crash_Military_Tm         int64  
 6   Deer_Nodeer               float64
 7   Distracted_Notdistracted  float64
 8   FAC                       object 
 9   First_Harmful_Event       object 
 10  FUN                       object 
 11  Hitrun_Not_Hitrun         float64
 12  LATITUDE                  float64
 13  Light_Condition           object 
 14  LONGITUDE                 float64
 15  Motor_Nonmotor            float64
 16  Passage                   float64
 17  Pdo_Crash                 int64  
 18  Pedage                    float64
 19  Roadway_Alignment         object 
 20  Roadway_Surface_Cond 

In [ ]:
crash_df_subset.shape

(155019, 102)

Take away fatal_crashes feature to use as label

In [ ]:
label = crash_df_subset['SEVERITY']

Numerical

In [ ]:
crash_num = crash_df_subset.drop(["Collision_Type","FAC", "First_Harmful_Event", "FUN", "Light_Condition", "Roadway_Alignment", "Roadway_Surface_Cond", "SEVERITY", "Weather_Condition"], axis=1)
crash_num.info(verbose='true')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 155019 entries, 758908 to 14714
Data columns (total 93 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Alcohol_Notalcohol        155019 non-null  float64
 1   Belted_Unbelted           155019 non-null  float64
 2   Bike_Nonbike              155019 non-null  float64
 3   Carspeedlimit             155019 non-null  float64
 4   Crash_Military_Tm         155019 non-null  int64  
 5   Deer_Nodeer               155019 non-null  float64
 6   Distracted_Notdistracted  155019 non-null  float64
 7   Hitrun_Not_Hitrun         155019 non-null  float64
 8   LATITUDE                  155019 non-null  float64
 9   LONGITUDE                 155019 non-null  float64
 10  Motor_Nonmotor            155019 non-null  float64
 11  Passage                   155019 non-null  float64
 12  Pdo_Crash                 155019 non-null  int64  
 13  Pedage                    155019 non-nul

Categorical

In [ ]:
#Categorical features
crash_cat = crash_df_subset[["Collision_Type","FAC", "First_Harmful_Event", "FUN", "Light_Condition", "Roadway_Alignment", "Roadway_Surface_Cond", "Weather_Condition"]]
crash_cat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 155019 entries, 758908 to 14714
Data columns (total 8 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   Collision_Type        155019 non-null  object
 1   FAC                   155019 non-null  object
 2   First_Harmful_Event   155019 non-null  object
 3   FUN                   155019 non-null  object
 4   Light_Condition       155019 non-null  object
 5   Roadway_Alignment     155019 non-null  object
 6   Roadway_Surface_Cond  155019 non-null  object
 7   Weather_Condition     155019 non-null  object
dtypes: object(8)
memory usage: 10.6+ MB


Create pipeline

In [ ]:
#imports
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

#Numerical pipeline
num_pipeline = Pipeline([
        ('std_scaler', StandardScaler()),
    ])

#Categorical encoder
cat_encoder = OneHotEncoder(sparse=False)

In [ ]:
from sklearn.compose import ColumnTransformer

num_attribs = list(crash_num) 
cat_attribs = list(crash_cat)

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

In [ ]:
crash_prepared = full_pipeline.fit_transform(crash_df_subset)
crash_prepared

array([[ 4.06621476, -0.19827962, -0.06662151, ...,  0.        ,
         0.        ,  0.        ],
       [-0.24592897, -0.19827962, -0.06662151, ...,  0.        ,
         0.        ,  0.        ],
       [-0.24592897, -0.19827962, -0.06662151, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.24592897, -0.19827962, -0.06662151, ...,  0.        ,
         0.        ,  0.        ],
       [-0.24592897, -0.19827962, -0.06662151, ...,  0.        ,
         0.        ,  0.        ],
       [-0.24592897, -0.19827962, -0.06662151, ...,  0.        ,
         0.        ,  0.        ]])

Split into the train and test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(crash_prepared, label, random_state=42)

In [ ]:
print(type(X_train))
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

<class 'numpy.ndarray'>
(116264, 178)
(38755, 178)
(116264,)


TA Layne reccomended to only train on 100k at first so I'm going to do another train test split

In [ ]:
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
sel_ = SelectFromModel(LogisticRegression(C=1, penalty='l1', solver='liblinear'))
sel_.fit(X_train, y_train)
sel_.get_support()

Logistic regression with lasso regularization

This was the output of the lasso, but I didn't want to run it again because it took a while so i copied the output here

In [ ]:
features = [  True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False,  True,  True, False,  True,  True,
        True, False,  True,  True, False,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True, False, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False, False,  True,  True, False,  True,
        True,  True,  True,  True, False, False, False]

In [ ]:
len(features)

178

In [ ]:
len(feature_labels_one_hot)

180

In [ ]:

X_train_mat = pd.DataFrame(X_train)

In [ ]:
sum(features) #401 trues

161

In [ ]:
selected_feat_train = X_train_mat.columns[features]

In [ ]:
selected_feat_train #matrix with 401 columns

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            162, 163, 164, 167, 168, 170, 171, 172, 173, 174],
           dtype='int64', length=161)

In [ ]:
X_test_mat = pd.DataFrame(X_test)
selected_feat_test = X_test_mat.columns[features]

In [ ]:
selected_feat_test

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            162, 163, 164, 167, 168, 170, 171, 172, 173, 174],
           dtype='int64', length=161)

In [ ]:
from sklearn.svm import SVC
svm = SVC(decision_function_shape='ovr', kernel = 'rbf')

Tune hyperparameters


In [ ]:
#Evaluate
from sklearn.metrics import mean_squared_error
import numpy as np
svm.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
import pickle
from sklearn.externals import joblib 
from google.colab import files

In [ ]:

  
# Save the model as a pickle in a file 
joblib.dump(svm, 'trainedModel.pkl') 

files.download('trainedModel.pkl')

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


NameError: ignored

SVM FIT

In [ ]:
from sklearn.metrics import classification_report 

In [ ]:
DATA_PATH = "/content/drive/My Drive/data/"
infile = open(DATA_PATH+'trainedModel.pkl')
savedModel = joblib.load('trainedModel.pkl')  

In [ ]:
savedModel

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
y_pred = savedModel.predict(X_train)

In [ ]:
target_names = ['MILD','MEDIUM', 'SEVERE']

In [ ]:
print(classification_report(y_train, y_pred, target_names=target_names))

              precision    recall  f1-score   support

        MILD       0.86      0.98      0.92     33976
      MEDIUM       1.00      1.00      1.00     75571
      SEVERE       0.68      0.22      0.33      6717

    accuracy                           0.95    116264
   macro avg       0.85      0.73      0.75    116264
weighted avg       0.94      0.95      0.94    116264



In [ ]:
#now try with the test set

In [ ]:
y_testPred = savedModel.predict(X_test)

In [ ]:
print(classification_report(y_test, y_testPred, target_names=target_names))

              precision    recall  f1-score   support

        MILD       0.86      0.97      0.91     11382
      MEDIUM       1.00      1.00      1.00     25114
      SEVERE       0.60      0.18      0.27      2259

    accuracy                           0.94     38755
   macro avg       0.82      0.72      0.73     38755
weighted avg       0.93      0.94      0.93     38755



Get support vectors and dual coefficients for analyisis in the report on feature importance

In [ ]:
print(savedModel.support_vectors_)

[[-0.24592897 -0.19827962 -0.06662151 ...  0.          0.
   0.        ]
 [-0.24592897 -0.19827962 -0.06662151 ...  0.          0.
   0.        ]
 [-0.24592897 -0.19827962 -0.06662151 ...  0.          0.
   0.        ]
 ...
 [-0.24592897 -0.19827962 -0.06662151 ...  0.          0.
   0.        ]
 [-0.24592897 -0.19827962 -0.06662151 ...  0.          0.
   0.        ]
 [-0.24592897 -0.19827962 -0.06662151 ...  0.          0.
   0.        ]]


In [ ]:
len(savedModel.support_vectors_[0])

178

In [ ]:
print(savedModel.support_vectors_)

[[-0.24592897 -0.19827962 -0.06662151 ...  0.          0.
   0.        ]
 [-0.24592897 -0.19827962 -0.06662151 ...  0.          0.
   0.        ]
 [-0.24592897 -0.19827962 -0.06662151 ...  0.          0.
   0.        ]
 ...
 [-0.24592897 -0.19827962 -0.06662151 ...  0.          0.
   0.        ]
 [-0.24592897 -0.19827962 -0.06662151 ...  0.          0.
   0.        ]
 [-0.24592897 -0.19827962 -0.06662151 ...  0.          0.
   0.        ]]


In [ ]:
savedModel.dual_coef_

array([[ 0.        ,  0.        ,  0.        , ..., -1.        ,
        -1.        , -1.        ],
       [ 1.        ,  0.66709561,  0.06710058, ..., -0.        ,
        -0.11331734, -0.        ]])

Run a bunch for hyperparameter tuning

In [ ]:
from sklearn.model_selection import GridSearchCV 
# defining parameter range 
param_grid = {'C': [0.1, 1],  
              'gamma': [1, 0.1], 
              'kernel': ['rbf']}  
  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3) 

In [ ]:
grid.fit(X_train,y_train)

In [ ]:
svm_grid_best = grid.best_estimator_

In [ ]:
# Save the model as a pickle in a file 
joblib.dump(svm_grid_best, 'trainedModeGrid.pkl') 

files.download('trainedModelGrid.pkl')

In [ ]:

files.download('trainedModelGrid.pkl')